# Inference Pipeline with Scikit-learn and Linear Learner
Typically a Machine Learning (ML) process consists of few steps: data gathering with various ETL jobs, pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm. 
In many cases, when the trained model is used for processing real time or batch prediction requests, the model receives data in a format which needs to pre-processed (e.g. featurized) before it can be passed to the algorithm. In the following notebook, we will demonstrate how you can build your ML Pipeline leveraging the Sagemaker Scikit-learn container and SageMaker Linear Learner algorithm & after the model is trained, deploy the Pipeline (Data preprocessing and Lineara Learner) as an Inference Pipeline behind a single Endpoint for real time inference and for batch inferences using Amazon SageMaker Batch Transform.

We will demonstrate this using the Abalone Dataset to guess the age of Abalone with physical features. The dataset is available from [UCI Machine Learning](https://archive.ics.uci.edu/ml/datasets/abalone); the aim for this task is to determine age of an Abalone (a kind of shellfish) from its physical measurements. We'll use Sagemaker's Scikit-learn container to featurize the dataset so that it can be used for training with Linear Learner.

### Table of contents
* [Preprocessing data and training the model](#training)
 * [Upload the data for training](#upload_data)
 * [Create a Scikit-learn script to train with](#create_sklearn_script)
 * [Create SageMaker Scikit Estimator](#create_sklearn_estimator)
 * [Batch transform our training data](#preprocess_train_data)
 * [Fit a LinearLearner Model with the preprocessed data](#training_model)
* [Inference Pipeline with Scikit preprocessor and Linear Learner](#inference_pipeline)
 * [Set up the inference pipeline](#pipeline_setup)
 * [Make a request to our pipeline endpoint](#pipeline_inference_request)
 * [Delete Endpoint](#delete_endpoint)

Let's first create our Sagemaker session and role, and create a S3 prefix to use for the notebook example.

In [46]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# S3 prefix
#bucket = sagemaker_session.default_bucket()
bucket = 'credito-poc'
prefix = 'test1'

In [47]:
sagemaker.__version__

'2.0.0'

# Preprocessing data and training the model <a class="anchor" id="training"></a>
## Downloading dataset <a class="anchor" id="download_data"></a>
SageMaker team has downloaded the dataset from UCI and uploaded to one of the S3 buckets in our account.

In [48]:
#!wget --directory-prefix=./abalone_data https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv

## Upload the data for training <a class="anchor" id="upload_data"></a>

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. We can use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [49]:
WORK_DIRECTORY = 'credito_data'

train_input = sagemaker_session.upload_data(
    path='{}/{}'.format(WORK_DIRECTORY, 'credito_limpo.csv'), 
    bucket=bucket,
    key_prefix='{}/{}'.format(prefix, 'train'))

In [50]:
train_input

's3://credito-poc/test1/train/credito_limpo.csv'

bggg## Create a Scikit-learn script to train with <a class="anchor" id="create_sklearn_script"></a>
To run Scikit-learn on Sagemaker `SKLearn` Estimator with a script as an entry point. The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
* SM_OUTPUT_DIR: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

Supposing two input channels, 'train' and 'test', were used in the call to the Chainer estimator's fit() method, the following will be set, following the format SM_CHANNEL_[channel_name]:

* SM_CHANNEL_TRAIN: A string representing the path to the directory containing data in the 'train' channel
* SM_CHANNEL_TEST: Same as above, but for the 'test' channel.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an argparse.ArgumentParser instance. For example, the script run by this notebook

###  script usado é credito_featurizer.py

## Create SageMaker Scikit Estimator <a class="anchor" id="create_sklearn_estimator"></a>

To run our Scikit-learn training script on SageMaker, we construct a `sagemaker.sklearn.estimator.sklearn` estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __framework_version__: Scikit-learn version you want to use for executing your model training code.
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training. __Note__: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.

To see the code for the SKLearn Estimator, see here: https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/sklearn

In [52]:
!python credito_featurizer.py --train credito_data  --model .  --output-data-dir  .

Traceback (most recent call last):
  File "credito_featurizer.py", line 81, in <module>
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
  File "/opt/conda/envs/sagemaker-soln/lib/python3.7/os.py", line 681, in __getitem__
    raise KeyError(key) from None
KeyError: 'SM_OUTPUT_DATA_DIR'


In [64]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = 'credito_featurizer.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    #train_instance_type="ml.c4.xlarge",
    instance_type = "ml.m5.large",
    sagemaker_session=sagemaker_session)


In [65]:
train_input

's3://credito-poc/test1/train/credito_limpo.csv'

In [66]:
sklearn_preprocessor.fit({'train': train_input})

2021-02-18 21:24:57 Starting - Starting the training job...
2021-02-18 21:24:59 Starting - Launching requested ML instances......
2021-02-18 21:26:22 Starting - Preparing the instances for training......
2021-02-18 21:27:12 Downloading - Downloading input data...
2021-02-18 21:27:47 Training - Downloading the training image..2021-02-18 21:28:04,043 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-02-18 21:28:04,046 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-18 21:28:04,066 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-02-18 21:28:04,491 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-18 21:28:04,515 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-18 21:28:04,539 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-18 21:28:04,555 sagemaker-

## Batch transform our training data <a class="anchor" id="preprocess_train_data"></a>
Now that our proprocessor is properly fitted, let's go ahead and preprocess our training data. Let's use batch transform to directly preprocess the raw data and store right back into s3.

In [67]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type= 'ml.m5.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv')

In [68]:
# Preprocess training input
transformer.transform(train_input, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)

..........................2021-02-18 21:32:47,657 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-02-18 21:32:47,659 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-02-18 21:32:47,660 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 0;

    keepalive_timeout 3;

    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }

    location / {
    

In [70]:

#transformer.wait()
preprocessed_train = transformer.output_path

## Fit a LinearLearner Model with the preprocessed data <a class="anchor" id="training_model"></a>
Let's take the preprocessed training data and fit a LinearLearner Model. Sagemaker provides prebuilt algorithm containers that can be used with the Python SDK. The previous Scikit-learn job preprocessed the raw Titanic dataset into labeled, useable data that we can now use to fit a binary classifier Linear Learner model.

For more on Linear Learner see: https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html

In [71]:
import boto3
from sagemaker.image_uris import retrieve
ll_image = retrieve('linear-learner', boto3.Session().region_name)

In [75]:
preprocessed_train

's3://sagemaker-us-east-2-785315722499/sagemaker-scikit-learn-2021-02-18-21-28-39-475'

In [ ]:
# import boto3
# import io
# s3 = boto3.client('s3')
# obj = s3.get_object(Bucket='sagemaker-us-east-2-785315722499', Key='sagemaker-scikit-learn-2021-02-18-21-28-39-475')
# df = pd.read_csv(io.BytesIO(obj['Body'].read()))

In [72]:
s3_ll_output_key_prefix = "ll_training_output"
s3_ll_output_location = 's3://{}/{}/{}/{}'.format(bucket, prefix, s3_ll_output_key_prefix, 'll_model')

ll_estimator = sagemaker.estimator.Estimator(
    ll_image,
    role, 
    instance_count=1, 
    instance_type='ml.m4.2xlarge',
    volume_size = 20,
    max_run = 3600,
    input_mode= 'File',
    output_path=s3_ll_output_location,
    sagemaker_session=sagemaker_session)

### incluir features incluindo a variavel a ser predita
ll_estimator.set_hyperparameters(feature_dim=47, predictor_type='binary_classifier', mini_batch_size=32)

ll_train_data = sagemaker.inputs.TrainingInput(
    preprocessed_train, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')

data_channels = {'train': ll_train_data}
ll_estimator.fit(inputs=data_channels, logs=True)

2021-02-18 21:36:24 Starting - Starting the training job...
2021-02-18 21:36:26 Starting - Launching requested ML instances......
2021-02-18 21:37:28 Starting - Preparing the instances for training......
2021-02-18 21:38:32 Downloading - Downloading input data
2021-02-18 21:38:32 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[02/18/2021 21:39:06 INFO 139729524229952] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_

UnexpectedStatusException: Error for Training job linear-learner-2021-02-18-21-36-23-994: Failed. Reason: ClientError: Rows 1-32 in file /opt/ml/input/data/train/credito_limpo.csv.out have different fields than the expected size 11.

# Serial Inference Pipeline with Scikit preprocessor and Linear Learner <a class="anchor" id="serial_inference"></a>


## Set up the inference pipeline <a class="anchor" id="pipeline_setup"></a>
Setting up a Machine Learning pipeline can be done with the Pipeline Model. This sets up a list of models in a single endpoint; in this example, we configure our pipeline model with the fitted Scikit-learn inference model and the fitted Linear Learner model. Deploying the model follows the same ```deploy``` pattern in the SDK.

In [23]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
linear_learner_model = ll_estimator.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, 
    role=role, 
    models=[
        scikit_learn_inferencee_model ,
        linear_learner_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

---------------!

## Make a request to our pipeline endpoint <a class="anchor" id="pipeline_inference_request"></a>

Here we just grab the first line from the test data (you'll notice that the inference python script is very particular about the ordering of the inference request data). The ```ContentType``` field configures the first container, while the ```Accept``` field configures the last container. You can also specify each container's ```Accept``` and ```ContentType``` values using environment variables.

We make our request with the payload in ```'text/csv'``` format, since that is what our script currently supports. If other formats need to be supported, this would have to be added to the ```output_fn()``` method in our entry point. Note that we set the ```Accept``` to ```application/json```, since Linear Learner does not support ```text/csv``` ```Accept```. The prediction output in this case is trying to guess the number of rings the abalone specimen would have given its other physical features; the actual number of rings is 10.

In [24]:
import sagemaker
from sagemaker import get_execution_role

from sagemaker.predictor import Predictor
#from sagemaker.predictor import predict
from sagemaker.serializers import CSVSerializer
sagemaker_session = sagemaker.Session()
endpoint_name = 'inference-pipeline-ep-2021-02-15-13-10-44'

payload = 'M, 0.44, 0.365, 0.125, 0.516, 0.2155, 0.114, 0.155'  ## resposta 10
#payload = 'M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15'         ## resposta 15

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer())

print(predictor.predict(payload))

b'{"predictions": [{"score": 9.528051376342773}]}'


In [23]:
runtime = boto3.client('runtime.sagemaker')
payload =  'M, 0.44, 0.365, 0.125, 0.516, 0.2155, 0.114, 0.155' 
endpoint_name = 'inference-pipeline-ep-2021-02-15-13-10-44'
# Send CSV text via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
# Unpack response
result = json.loads(response['Body'].read().decode())
result

{'predictions': [{'score': 9.528051376342773}]}

## Delete Endpoint <a class="anchor" id="delete_endpoint"></a>
Once we are finished with the endpoint, we clean up the resources!

In [ ]:
sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)